"""
Script: SEC 13F Filings Database Pipeline

Description:
    This script connects to a local PostgreSQL database and sets up 
    the schema to store SEC 13F filings and their holdings. It then 
    retrieves 13F filings from the SEC API using pagination, parses 
    the data, and writes it into the database.

Main Components:
    1. Database Connection:
        - Uses psycopg2 to connect to a PostgreSQL instance.
        - Ensures autocommit is disabled so transactions are committed explicitly.
        - Provides initial sanity check by printing the database version.

    2. Table Creation:
        - Creates three tables: filings, holdings, and holding_infos.
        - Ensures holdings columns (`value` and `shares`) are stored as BIGINT 
          to handle large volumes.

    3. Data Retrieval (get_13f_filings):
        - Queries the SEC API for 13F-HR filings (excluding amended 13F-HR/A).
        - Fetches data from January 1, 2000 onwards.
        - Supports pagination through `start` and `size` parameters.
        - Returns filings in JSON format.

    4. Data Insertion (save_to_db):
        - Inserts filings into the `filings` table, skipping duplicates.
        - Inserts each holding into the `holdings` table.
        - Handles missing fields safely (e.g., `CIK`, `putCall`, `titleOfClass`).

    5. Main Loop (fill_database):
        - Iteratively fetches filings from the SEC API in batches of 100.
        - Saves each batch into the database.
        - Prints a dot (.) for progress.
        - After all batches are processed, prints the total number of holdings.

Input:
    - Database connection credentials (host, port, dbname, user, password).
    - SEC API key (passed into QueryApi).
    - Optional pagination parameters: start index (default=0) and size (default=100).

Output:
    - Prints PostgreSQL server version at the beginning.
    - Prints progress dots while ingesting filings.
    - Prints total number of holdings in the database at the end.
    - Commits data into the database tables `filings` and `holdings`.

Example Output:
    PostgreSQL database version:
    ('PostgreSQL 16.3, compiled by Visual C++ build 1914, 64-bit',)
    ....
    Total holdings in DB: 452300
    Done
"""


In [ ]:
# fill-database.py
import sys
import os
import psycopg2
from psycopg2.extras import execute_batch
import pandas as pd
from sec_api import QueryApi

queryApi = QueryApi(api_key='******')


#1. Database Connection:
        - Uses psycopg2 to connect to a PostgreSQL instance.
        - Ensures autocommit is disabled so transactions are committed explicitly.
        - Provides initial sanity check by printing the database version.


In [ ]:


conn = psycopg2.connect(
    host="localhost",    # database server address (localhost means same machine)
    port=5432,              # default PostgreSQL port 
    dbname="sec13f",         # name of the database to connect to
    user="postgres",            # username for authentication
    password="****"        # password (this is the data base password)
)

# Disable autocommit so changes must be committed manually
conn.autocommit = False

# Create a cursor object to execute SQL queries
cur = conn.cursor()


In [ ]:
# create a cursor
cur = conn.cursor()

# execute a statement
cur.execute('SELECT version()')

# display the PostgreSQL database server version
db_version = cur.fetchone()

print('PostgreSQL database version:')
print(db_version)

# close the communication with the PostgreSQL
cur.close()

##2. Table Creation:
        - Creates three tables: filings, holdings, and holding_infos.
        - Ensures holdings columns (`value` and `shares`) are stored as BIGINT 
          to handle large volumes.

In [ ]:
def create_tables():
    
     # Create a cursor object to interact with the database
    cur = conn.cursor()

    # Define SQL commands to create the necessary tables
    create_table_commands = (
        # Table to store information about each filing
        """
            CREATE TABLE filings (
                filing_id varchar(255) PRIMARY KEY,    -- unique identifier for each filing
                cik int,                                -- central index key of the filer
                filer_name varchar(255),                -- name of the filer
                period_of_report date                   -- reporting period
            )
        """,
        """
            CREATE TABLE holdings (
                filing_id varchar(255),                  -- reference to the filing ID
                name_of_issuer varchar(255),             -- name of the issuer of the security
                cusip varchar(255),                      -- CUSIP identifier of the security
                cik text,                                -- CIK of the issuer
                title_of_class varchar(255),             -- class of the security
                value bigint,                            -- value of the holding
                shares int,                               -- value of the holding  
                put_call varchar(255)                    -- option type, if applicable
            )
        """, 
        """
            CREATE TABLE holding_infos (
                cusip varchar(255),                         -- CUSIP identifier (link to holdings table)                  
                security_name varchar(255),                 -- name of the security
                ticker varchar(50),                         -- ticker symbol
                exchange_code varchar(10),                  -- exchange code 
                security_type varchar(50)                   -- type of security (e.g., stock, bond) 
            )
        """)

    # create table one by one
    for command in create_table_commands:
        cur.execute(command)
    
    # close cursor
    cur.close()
    
    # make the changes to the database persistent
    conn.commit()

In [ ]:
# Call the function to create the necessary tables in the database
create_tables()

###
"""
    Ensure 'value' and 'shares' columns in the 'holdings' table use BIGINT.
    Reason:
        13F holdings can have very large numeric values; BIGINT (8-byte signed)
        prevents overflow that could occur with INT/INTEGER.
    Behavior:
        - Attempts to ALTER COLUMN types inside a managed cursor context.
        - If an ALTER fails (e.g., column already BIGINT, missing column, or
          incompatible data), prints a warning and proceeds with the next change.
        - Commits any successful alterations at the end.
    """

In [ ]:
def ensure_bigint_columns():
     # Use a context manager so the cursor is closed automatically
    with conn.cursor() as cur:
        try:
            # Change 'value' column to BIGINT; participates in the current transaction
            cur.execute("""
                ALTER TABLE holdings
                ALTER COLUMN value TYPE BIGINT
            """)
        except Exception as e:
             # Non-fatal: log a warning and continue (e.g., already BIGINT or constraint issue)
            print("Warning: Could not alter 'value' column to BIGINT:", e)

        try:
             # Change 'shares' column to BIGINT for large share counts
            cur.execute("""
                ALTER TABLE holdings
                ALTER COLUMN shares TYPE BIGINT
            """)
        except Exception as e:
            # Non-fatal: log a warning and continue
            print("Warning: Could not alter 'shares' column to BIGINT:", e)
    
    # Commit successful alterations (if any). Note: if one ALTER failed and the other
    # succeeded, this will persist only the successful change(s).
    conn.commit()

In [ ]:
ensure_bigint_columns()

 3. Data Retrieval (get_13f_filings):
        - Queries the SEC API for 13F-HR filings (excluding amended 13F-HR/A).
        - Fetches data from January 1, 2000 onwards.
        - Supports pagination through `start` and `size` parameters.
        - Returns filings in JSON format.

In [ ]:
def get_13f_filings(start=0, size=100):
    # Print a message showing which batch is being requested
    print(f"Getting next 13F batch starting at {start}")
    
    query = {
      # Define the query for the EDGAR API
      # - Only retrieve 13F-HR filings (exclude amended 13F-HR/A)
      # - Filter filings from January 1, 2000 onwards
      # - Paginate results with 'from' (start index) and 'size' (batch size)
      # - Sort results by filing date in descending order
      "query": { "query_string": { 
          "query": "formType:\"13F-HR\" AND NOT formType:\"13F-HR/A\" AND periodOfReport:{ 2000-01-01 TO *}" 
        } },
      "from": start,
      "size": size,
      "sort": [{ "filedAt": { "order": "desc" } }]
    }
    # Send the query to the EDGAR API
    response = queryApi.get_filings(query)

    # Return only the list of filings from the API response
    return response['filings']

# Fetch the 10 most recent 13F filings (default size=100, here explicitly set to 10 if needed)
filings_batch = get_13f_filings()

# Load all holdings of the first 13F filing into a pandas DataFrame
# This flattens the nested JSON structure into tabular format
holdings_example = pd.json_normalize(filings_batch[0]['holdings'])

4. Data Insertion (save_to_db):
        - Inserts filings into the `filings` table, skipping duplicates.
        - Inserts each holding into the `holdings` table.
        - Handles missing fields safely (e.g., `CIK`, `putCall`, `titleOfClass`).


In [ ]:
def save_to_db(filings):
    # Create a cursor object to execute SQL commands
    cur = conn.cursor()

     # Loop over each filing in the list of filings

    for filing in filings:
        # Skip this filing if it has no holdings
        if len(filing["holdings"]) == 0:
            continue

        # SQL insert commands for filings and holdings tables
        # Note: filings use ON CONFLICT (filing_id) DO NOTHING to avoid duplicates

        insert_commands = (
            """
                INSERT INTO filings (
                    filing_id, 
                    cik,
                    filer_name,
                    period_of_report
                ) 
                
                VALUES (%s, %s, %s, %s)
                ON CONFLICT (filing_id) DO NOTHING
            """,
            """
                INSERT INTO holdings (
                    filing_id, 
                    name_of_issuer,
                    cusip,
                    cik,
                    title_of_class,
                    value,
                    shares,
                    put_call
                ) 
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            """,
        )

        # Values for inserting into the filings table

        filing_values = (
            filing["id"],                                   # Unique filing identifier
            filing["cik"],                                  # Central Index Key
            filing["companyName"].upper(),                  # Convert company name to uppercase
            filing["periodOfReport"],                        # Reporting period
        )

          # Insert filing data into the filings table
        cur.execute(insert_commands[0], filing_values)

          # Loop through all holdings associated with this filing
        for holding in filing["holdings"]:

            # Prepare values for inserting into the holdings table
            holding_values = (
                filing["id"],                                                               # Foreign key to link back to filings
                holding["nameOfIssuer"].upper(),                                            # Issuer name, converted to uppercase
                holding["cusip"],                                                           # Security identifier (CUSIP)
                holding.get("cik", ""),                                                     # Some holdings may not have a CIK
                holding["titleOfClass"] if "titleOfClass" in holding else "",               # Security class
                holding["value"],                                                           # Value of the holding
                holding["shrsOrPrnAmt"]["sshPrnamt"],                                       # Number of shares
                holding["putCall"] if "putCall" in holding else ""                          # Option type (if any)
            )

             # Insert holding data into the holdings table
            cur.execute(insert_commands[1], holding_values)

     # Close cursor and commit changes to the database    
    cur.close()
    conn.commit()
    

5. Main Loop (fill_database):
        - Iteratively fetches filings from the SEC API in batches of 100.
        - Saves each batch into the database.
        - Prints a dot (.) for progress.
        - After all batches are processed, prints the total number of holdings.


In [ ]:

def fill_database():
   
    start = 0  # Starting index for pagination
    while True:
        # Starting index for pagination
        filings = get_13f_filings(start=start)

        # If no filings are returned, exit the loop (end of data)
        if not filings:
            break

        # Save the retrieved filings into the database
        save_to_db(filings)
        print('.')

         # Move the starting point forward by 100 (the page size)
        start += 100

    # Sanity check: count total rows in the 'holdings' table
    with conn.cursor() as cur:
        cur.execute("SELECT COUNT(*) FROM public.holdings")
        print("Total holdings in DB:", cur.fetchone()[0])


         # Indicate the process is complete
    print("Done")

In [ ]:

 #Undo all uncommitted changes in the current transaction.
 # This is typically used inside an exception handler 
 # to ensure the database is not left in an inconsistent state 
# if an error occurs during insert/update operations.

conn.rollback()

In [ ]:
# Call the main function to fetch all 13F filings via pagination,
 # insert them into the database, and print a progress indicator 
# along with a final count of total holdings stored.
fill_database()

# Open a cursor; run SELECT COUNT(*) FROM holdings; fetch the single result; print the count; close the cursor.

In [ ]:
# Create a new cursor object to interact with the database
cur = conn.cursor()

# Execute a SQL query to count the total number of rows in the 'holdings' table
cur.execute("SELECT count(*) as COUNTER FROM holdings")

# Fetch the first row of the result (a single value in this case)
counter = cur.fetchone()

# Print the count (first element of the result tuple)
print(counter[0])

# Close the cursor to free up database resources
cur.close()